In [ ]:
#Input Data
#Uncompress the input variables .tar archieve 
#!tar xf var5.tar #Use this instead for five-variables models
!tar xf var3.tar
#!tar xf var2.tar #Use this instead for two-variables models

#Uncompress the Land Use/Land Cover .tar archieve (Contains one file)
!tar xf LULC.tar
#Uncompress the Rainfall  .tar archieve
!tar xf Rainfall.tar


#Output Data
#Uncompress the Soil Moisture  .tar archieve
!tar xf NWM_OUT_SM.tar

In [ ]:
#Runthis
from osgeo import gdal
import os
import numpy as np

#num_of_variables = 5  #Use this instead for five-variables models
num_of_variables = 3    
#num_of_variables = 2  #Use this instead for two-variables models

number_of_sequences = (365 + 153)*8 - 2

number_of_all_possible_sequences = (365 + 153)*24 -8

channels = num_of_variables + 2

#Dimensions of the inut
rows =65 
columns = 55

#Populate a list of input variables files
filesList = sorted(os.listdir('./var'+ str(num_of_variables)))


#filesList = filesList[0:-(num_of_variables*4)]                   #Use this instead for exclusive convLSTM sequence = 5 
filesList = filesList[num_of_variables*2:-(num_of_variables*4)]   #Use this instead for exclusive convLSTM sequence = 3 
#filesList = filesList[num_of_variables*1:-(num_of_variables*3)]  #Use this instead for Inclusive convLSTM sequence = 5 
#filesList = filesList[num_of_variables*3:-(num_of_variables*3)]  #Use this instead for Inclusive convLSTM sequence = 3

#Populate a list of rainfall files
RFFilesList = sorted(os.listdir('./Rainfall'))

#RFFilesList = RFFilesList[1:-3] #Use this instead for exclusive convLSTM sequence = 5 
RFFilesList = RFFilesList[3:-3]  #Use this instead for exclusive convLSTM sequence = 3 
#RFFilesList = RFFilesList[2:-2] #Use this instead for inclusive convLSTM sequence = 5 
#RFFilesList = RFFilesList[4:-2] #Use this instead for inclusive convLSTM sequence = 3


#Generate a complete list of inputs including Land use / Land Cover
NewCompletelist = []
for i in range(0,len(filesList),num_of_variables):
    for j in range(0,num_of_variables):
        NewCompletelist.append(filesList[i+j])
    NewCompletelist.append(RFFilesList[i//num_of_variables])
    NewCompletelist.append('LULC.tif_1km.tif')


#A function to load input data
def read_input_files(mylist):
    finallist=[]
    count =0
    for fn in (mylist):
        count =count +1
        if 'wrfsfcf' in fn:
            raster = gdal.Open(os.path.join('./var' + str(num_of_variables), fn))
        elif 'GaugeCorr' in fn:
            raster = gdal.Open(os.path.join('./Rainfall', fn))
        else:
            raster = gdal.Open(os.path.join('./LULC', fn))
        if raster is None:
            print ('Unable to open %s')
            break
        band = raster.GetRasterBand(1)
        array = band.ReadAsArray()
        finallist.append(array)   
    return finallist

#Allmerged contains all input data 
Allmerged = np.array(read_input_files(NewCompletelist))

#Allmerged reshaped in the form: (number_of_entries, channels, 65, 55)
length =(Allmerged.shape[0])//(channels)
Allmerged = Allmerged.reshape((length,(channels ),*Allmerged.shape[-2:]))

#Reshaping the input to be in the form: (number_of_entries, 65, 55, channels)
Allmerged= np.moveaxis(Allmerged, 1, -1)
print(Allmerged.shape)

#Generate a list of input sequences
new_final_sequences_list =[]  

#Use the following loop for sequence = 3
for i in range(0,number_of_all_sequences): 
    newarray=np.stack((Allmerged [i],Allmerged [i+1],Allmerged [i+2]),axis =0)
    new_final_sequences_list.append(newarray)

#Instead Use the following loop for sequence = 5
# for i in range(0,number_of_all_sequences): 
#     newarray=np.stack((Allmerged [i],Allmerged [i+1],Allmerged[i+2],Allmerged[i+3],Allmerged[i+4]),axis =0)
#     new_final_sequences_list.append(newarray)


#Convert the input sequences list into array
X = np.array(new_final_sequences_list)

#Performing Min-Max scalling for the input
for index in range(0, channels):
    max = np.max(X[:,:,:,:,index])
    min = np.min(X[:,:,:,:,index])
    X[:,:,:,:,index] = (X[:,:,:,:,index] - min)/(max-min)

print(X.shape) 

In [ ]:
#A function to read and generate output filenames "following the same existing naming conventions"
def read_generate_output_files(foldername):
    finallistnames = []
   
    path = './' + foldername
    
    for fn in sorted(os.listdir(path)):
        splitted = fn.split('.',1)
        first_split = splitted[0]
        second_split = splitted[1]
        
        file_name_first_part = first_split [0:8]
        file_name_second_part = first_split [8:10]
        
        finallistnames.append(fn)
        
        base = int(file_name_second_part)
        base_next = base + 1
        base_next_next = base + 2
        if base_next < 10:
            base_next = "0"+ str(base_next)
            
        if base_next_next < 10:
            base_next_next = "0"+ str(base_next_next)
        
        file_2 =  file_name_first_part + str(base_next) + "00." + second_split
        finallistnames.append(file_2)
        
        file_3 =  file_name_first_part + str(base_next_next) + "00." + second_split
        finallistnames.append(file_3)   
    return finallistnames

#Read and generate soil moisture output file names
SMFileNames = read_generate_output_files('NWM_OUT_SM')
SMFileNames = SMFileNames[6:-2]

In [ ]:
#Load a previously saved model
from keras.models import load_model

model = load_model('ConvLSTM_model.h5')

print(model.summary())

In [ ]:
#Generate predictions to the whole dataset 

predictions = model.predict(X)

#Defining a function to convert a numpy array to raster format

import  osr

def array2raster(newRasterfn,rasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    cols = array.shape[1]
    rows = array.shape[0]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

#create a folder to save the predictions in raster format
!mkdir predicted

for i in range (0,len(SMFileNames)):
  #read original output filename
  originalfilename = SMFileNames[i]

  sample_prediction = np.squeeze(predictions[i]) 

  #Generate predicted output filename
  filename = originalfilename [0:-3] + "predicted"+ ".tif"
  predictedpath = "predicted/"+ filename
  #save theoutput prediction in tif format
  array2raster(predictedpath,'/content/LULC/LULC.tif_1km.tif',sample_prediction)

#Archieve all predictions in one .tar file
!tar cf  All_sequences_predicted_convLSTM.tar predicted